In [1]:
import pandas as pd
import csv
import json
import ast
import numpy as np
import copy
import altair

In [2]:
movieData = pd.read_csv("movies_metadata.csv")

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# variable to hold all of the genres
uniqueGenres = set()

# matches each movie title with associated genres
titleGenres = {}

#converts the strings in movie metadata to lists
movieData["genres"] = movieData.genres.apply(lambda x: ast.literal_eval(str(x)))

for link in movieData["genres"]:
    genreList = link
    for genre in genreList:
        uniqueGenres.add(genre["name"])
        
#print(uniqueGenres)


genresData = {}
for genre in uniqueGenres:
    genresData[genre] = 0

totalRevenue = 0
for i in range(0, len(movieData["original_title"])):
    
    # when cleaning the data we ran into weird errors so we just checked if null inside the loop
    if np.isnan(movieData["revenue"][i]):
        continue
    
    # temporary list of genres associated with that movie
    genreList1 = movieData["genres"][i]
    #print(movieData["revenue"][i])
    # adds this list to a dict associating the genres with titles for later
    titleGenres[movieData["original_title"][i]] = genreList1    
    
    # calculate total revenue overall
    boxOffice = float(movieData["revenue"][i])
    totalRevenue += boxOffice
    
    # calculate total box office for each genre
    for genre in genreList1:
        #print(genre["name"])
        genresData[genre["name"]] += boxOffice

# check if genres were labeled correctly
print(genresData)
# create copy in order to avoid modification while iterating
genresCopy = copy.deepcopy(genresData)

# delete all genres with zero revenue
for entry in genresCopy:
    if genresData[entry] == 0:
        del genresData[entry]
        
# see which genres remain
genresRatios = {}

# find the box office percentage taken up by movies tagged with each genre
for key in genresData.keys():
    genresRatios[key] = [(genresData[key]/totalRevenue)*100]
genresRatios1 = pd.DataFrame.from_dict(genresRatios, orient="columns")
genresRatios1

{'Foreign': 291633311.0, 'Music': 13370292367.0, 'Aniplex': 0, 'GoHands': 0, 'Western': 5122498884.0, 'Drama': 160844199830.0, 'Romance': 73506661600.0, 'Vision View Entertainment': 0, 'Adventure': 200119343481.0, 'Odyssey Media': 0, 'Documentary': 1449528578.0, 'Horror': 30837094673.0, 'Thriller': 129821552609.0, 'Animation': 67530135617.0, 'The Cartel': 0, 'History': 14902198420.0, 'War': 15910458288.0, 'Mardock Scramble Production Committee': 0, 'Carousel Productions': 0, 'Sentai Filmworks': 0, 'Fantasy': 104034711338.0, 'BROSTA TV': 0, 'Mystery': 34754614989.0, 'Pulser Productions': 0, 'TV Movie': 42000000.0, 'Rogue State': 0, 'Telescene Film Group Productions': 0, 'Action': 201456461294.0, 'Crime': 63419676943.0, 'Family': 107235185780.0, 'Science Fiction': 97915233082.0, 'Comedy': 166897270043.0}


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy
0,0.05723,2.623802,1.005245,31.564259,14.425036,39.271661,0.284457,6.051509,25.476338,13.252195,2.924425,3.122287,20.415897,6.820287,0.008242,39.534058,12.445554,21.043962,19.215003,32.752121


In [4]:
# find genre ratios by year
yearsRatios = {}
def genreExtractor(arg):
    genreList = []
    for genre in arg:
        genreList.append(genre["name"])
    return genreList
# clean up weird release dates 
movieData["release_date"] = movieData.release_date.apply(lambda x: "00000000" if len(str(x)) < 7 else x)
movieData["release_year"] = movieData["release_date"].apply(lambda x: str(x)[0:4])
movieData["genre_names"] = movieData["genres"].apply(lambda x: genreExtractor(x))
yearRevenues = movieData.groupby("release_year")["revenue"].sum()
print(yearRevenues)

release_year
0000    1.000000e+01
1874    0.000000e+00
1878    0.000000e+00
1883    0.000000e+00
1887    0.000000e+00
            ...     
2015    2.881936e+10
2016    3.049788e+10
2017    1.507589e+10
2018    0.000000e+00
2020    0.000000e+00
Name: revenue, Length: 136, dtype: float64


In [5]:
# starting in 1994 since this is the first year where #movies > 500
print(genresData.keys())
totalRatios = pd.DataFrame(columns=list(genresData.keys()))
for i in range(1994, 2017):
    newDict = {}
    yearsRatios[i] = {}
    for genre in genresData.keys():
        yearsRatios[i][genre] = 0.0
        if (i == 1994):
            newDict[genre] = 0
    if (i == 1994):
        newDict["total"] = 0
        totalRatios = totalRatios.append(newDict, ignore_index = True)
# preserves keys 
for i in range(0, len(movieData["release_year"])):
    year = movieData["release_year"][i]
    if int(year) not in yearsRatios.keys():
        continue
    for genre in movieData["genre_names"][i]:
        yearsRatios[int(year)][genre] += movieData["revenue"][i]
        totalRatios[genre] += movieData["revenue"][i]
    totalRatios["total"] += movieData["revenue"][i]
        
for year in yearsRatios.keys():
    for genre in yearsRatios[year].keys():
        yearsRatios[year][genre] = yearsRatios[year][genre]/yearRevenues[str(year)]
yearsRatios1 = pd.DataFrame.from_dict(yearsRatios).transpose()
totalRatios

dict_keys(['Foreign', 'Music', 'Western', 'Drama', 'Romance', 'Adventure', 'Documentary', 'Horror', 'Thriller', 'Animation', 'History', 'War', 'Fantasy', 'Mystery', 'TV Movie', 'Action', 'Crime', 'Family', 'Science Fiction', 'Comedy'])


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,...,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,total
0,2.76814e+08,9.10807e+09,3.05116e+09,1.28111e+11,5.76734e+10,1.69304e+11,1.39063e+09,2.35122e+10,1.05152e+11,6.04515e+10,...,1.07054e+10,9.00043e+10,2.90995e+10,4.2e+07,1.67286e+11,4.94573e+10,9.08656e+10,8.26038e+10,1.34958e+11,4.177191e+11


In [6]:
totalRatios1 = copy.deepcopy(totalRatios)
totalRatios1

,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,...,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,total
0,2.76814e+08,9.10807e+09,3.05116e+09,1.28111e+11,5.76734e+10,1.69304e+11,1.39063e+09,2.35122e+10,1.05152e+11,6.04515e+10,...,1.07054e+10,9.00043e+10,2.90995e+10,4.2e+07,1.67286e+11,4.94573e+10,9.08656e+10,8.26038e+10,1.34958e+11,4.177191e+11


In [7]:
for i in range(0, len(totalRatios1["Fantasy"])):
    for genre in genresData.keys():
        totalRatios1[genre][i] /= totalRatios1["total"][i]
totalRatios1

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,...,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,total
0,0.000662679,0.0218043,0.00730433,0.306693,0.138067,0.405305,0.00332911,0.0562872,0.251729,0.144718,...,0.0256282,0.215466,0.0696628,0.000100546,0.400474,0.118398,0.217528,0.19775,0.323083,4.177191e+11


In [8]:
oscarNoms = pd.read_csv("oscarNoms.csv")
oscarNomsCleaned = oscarNoms[5:len(oscarNoms)]
oscarNomsCleaned

,Documentary (Feature),Art Direction,Sound Mixing,Foreign Language Film,Makeup and Hairstyling,Music (Original Score),Sound,Special Achievement Award,Writing (Adapted Screenplay),Jean Hersholt Humanitarian Award,...,Actress in a Leading Role,Film Editing,Irving G. Thalberg Memorial Award,Music (Original Song),Sound Effects Editing,Music (Original Musical or Comedy Score),Special Achievement Award (Visual Effects),Writing (Screenplay Written Directly for the Screen),Actor in a Leading Role,Makeup
5,"['Maya Lin: A Strong Clear Vision', 'Complaint...","['The Madness of King George', 'Bullets over B...",NaN,"['Burnt by the Sun', 'Before the Rain', 'Eat D...",NaN,"['The Lion King', 'Forrest Gump', 'Interview w...","['Speed', 'Clear and Present Danger', 'Forrest...",NaN,NaN,[''],...,"['Blue Sky', 'Nell', 'Tom & Viv', 'Little Wome...","['Forrest Gump', 'Hoop Dreams', 'Pulp Fiction'...",[''],"['The Lion King', 'The Lion King', 'The Lion K...","['Speed', 'Clear and Present Danger', 'Forrest...",NaN,NaN,"['Pulp Fiction', 'Bullets over Broadway', 'Fou...","['Forrest Gump', 'The Shawshank Redemption', '...","['Ed Wood', 'Forrest Gump', ""Mary Shelley's Fr..."
6,"['Anne Frank Remembered', 'The Battle over Cit...","['Restoration', 'Apollo 13', 'Babe', 'A Little...",NaN,"[""Antonia's Line"", 'All Things Fair', 'Dust of...",NaN,NaN,"['Apollo 13', 'Batman Forever', 'Braveheart', ...",['Toy Story'],NaN,NaN,...,"['Dead Man Walking', 'Leaving Las Vegas', 'Cas...","['Apollo 13', 'Babe', 'Braveheart', 'Crimson T...",NaN,"['Pocahontas', 'Dead Man Walking', 'Don Juan D...","['Braveheart', 'Batman Forever', 'Crimson Tide']","['Pocahontas', 'The American President', 'Sabr...",NaN,"['The Usual Suspects', 'Braveheart', 'Mighty A...","['Leaving Las Vegas', ""Mr. Holland's Opus"", 'N...","['Braveheart', 'My Family, Mi Familia', 'Roomm..."
7,"['When We Were Kings', 'The Line King: The Al ...","['The English Patient', 'The Birdcage', 'Evita...",NaN,"['Kolya', 'A Chef in Love', 'The Other Side of...",NaN,NaN,"['The English Patient', 'Evita', 'Independence...",NaN,NaN,NaN,...,"['Fargo', 'Secrets & Lies', ""Marvin's Room"", '...","['The English Patient', 'Evita', 'Fargo', 'Jer...",[''],"['Evita', 'Up Close and Personal', 'One Fine D...","['The Ghost and the Darkness', 'Daylight', 'Er...","['Emma', 'The First Wives Club', 'The Hunchbac...",NaN,"['Fargo', 'Jerry Maguire', 'Lone Star', 'Secre...","['Shine', 'Jerry Maguire', 'The English Patien...","['The Nutty Professor', 'Ghosts of Mississippi..."
8,"['The Long Way Home', 'Ayn Rand: A Sense of Li...","['Titanic', 'Gattaca', 'Kundun', 'L.A. Confide...",NaN,"['Character', 'Beyond Silence', 'Four Days in ...",NaN,NaN,"['Titanic', 'Air Force One', 'Con Air', 'Conta...",NaN,NaN,NaN,...,"['As Good as It Gets', 'The Wings of the Dove'...","['Titanic', 'Air Force One', 'As Good as It Ge...",NaN,"['Titanic', 'Hercules', 'Con Air', 'Anastasia'...","['Titanic', 'Face/Off', 'The Fifth Element']","['The Full Monty', 'Anastasia', 'As Good as It...",NaN,"['Good Will Hunting', 'As Good as It Gets', 'B...","['As Good as It Gets', 'Good Will Hunting', 'T...","['Men in Black', 'Mrs. Brown', 'Titanic']"
9,"['The Last Days', 'Dancemaker', 'The Farm: Ang...","['Shakespeare in Love', 'Elizabeth', 'Pleasant...",NaN,"['Life Is Beautiful', 'Central Station', 'Chil...",NaN,NaN,"['Saving Private Ryan', 'Armageddon', 'The Mas...",NaN,NaN,NaN,...,"['Shakespeare in Love', 'Elizabeth', 'Central ...","['Saving Private Ryan', 'Life Is Beautiful', '...",[''],"['The Prince of Egypt', 'Armageddon', 'Quest f...","['Saving Private Ryan', 'Armageddon', 'The Mas...","['Shakespeare in Love', ""A Bug's Life"", 'Mulan...",NaN,"['Shakespeare in Love', 'Bulworth', 'Life Is B...","['Life Is Beautiful', 'Saving Private Ryan', '...","['Elizabeth', 'Saving Private Ryan', 'Shakespe..."
10,"['One Day in September', 'Buena Vista Social C...","['Sleepy Hollow', 'Anna and the King', 'The Ci...",NaN,"['All about My Mother', 'Caravan', 'East-West'...",NaN,"['The Red Vi

In [9]:
pd.set_option('display.max_columns', None)
oscarNomsCleaned1 = copy.deepcopy(oscarNomsCleaned)
oscarNomsCleaned1 = oscarNomsCleaned1.fillna(0)
oscarNomsCleaned1.applymap(lambda x: str(x).strip("[]").split(","))
oscarNomsCleaned1 = oscarNomsCleaned1.drop(columns=["Jean Hersholt Humanitarian Award", "Irving G. Thalberg Memorial Award"])
oscarNomsCleaned1



,Documentary (Feature),Art Direction,Sound Mixing,Foreign Language Film,Makeup and Hairstyling,Music (Original Score),Sound,Special Achievement Award,Writing (Adapted Screenplay),Visual Effects,Actress in a Supporting Role,Writing (Screenplay Based on Material from Another Medium),Best Picture,Sound Editing,Writing (Original Screenplay),Short Film (Live Action),Animated Feature Film,Costume Design,Short Film (Animated),Directing,Production Design,Documentary (Short Subject),Cinematography,Actor in a Supporting Role,Writing (Screenplay Based on Material Previously Produced or Published),Music (Original Dramatic Score),Actress in a Leading Role,Film Editing,Music (Original Song),Sound Effects Editing,Music (Original Musical or Comedy Score),Special Achievement Award (Visual Effects),Writing (Screenplay Written Directly for the Screen),Actor in a Leading Role,Makeup
5,"['Maya Lin: A Strong Clear Vision', 'Complaint...","['The Madness of King George', 'Bullets over B...",0,"['Burnt by the Sun', 'Before the Rain', 'Eat D...",0,"['The Lion King', 'Forrest Gump', 'Interview w...","['Speed', 'Clear and Present Danger', 'Forrest...",0,0,"['Forrest Gump', 'The Mask', 'True Lies']","['Bullets over Broadway', 'Tom & Viv', 'The Ma...",0,"['Forrest Gump', 'Four Weddings and a Funeral'...",0,0,"[""Franz Kafka's It's a Wonderful Life"", 'Trevo...",0,"['The Adventures of Priscilla, Queen of the De...","[""Bob's Birthday"", 'The Big Story', 'The Janit...","['Forrest Gump', 'Bullets over Broadway', 'Pul...",0,"['A Time for Justice', 'Blues Highway', '89mm ...","['Legends of the Fall', 'Forrest Gump', 'Red',...","['Ed Wood', 'Pulp Fiction', 'Bullets over Broa...","['Forrest Gump', 'The Madness of King George',...",0,"['Blue Sky', 'Nell', 'Tom & Viv', 'Little Wome...","['Forrest Gump', 'Hoop Dreams', 'Pulp Fiction'...","['The Lion King', 'The Lion King', 'The Lion K...","['Speed', 'Clear and Present Danger', 'Forrest...",0,0,"['Pulp Fiction', 'Bullets over Broadway', 'Fou...","['Forrest Gump', 'The Shawshank Redemption', '...","['Ed Wood', 'Forrest Gump', ""Mary Shelley's Fr..."
6,"['Anne Frank Remembered', 'The Battle over Cit...","['Restoration', 'Apollo 13', 'Babe', 'A Little...",0,"[""Antonia's Line"", 'All Things Fair', 'Dust of...",0,0,"['Apollo 13', 'Batman Forever', 'Braveheart', ...",['Toy Story'],0,"['Babe', 'Apollo 13']","['Mighty Aphrodite', 'Nixon', 'Apollo 13', 'Ge...",0,"['Braveheart', 'Apollo 13', 'Babe', 'The Postm...",0,0,"['Lieberman in Love', 'Brooms', 'Duke of Groov...",0,"['Restoration', 'Braveheart', 'Richard III', '...","['A Close Shave', 'The Chicken from Outer Spac...","['Braveheart', 'Babe', 'Dead Man Walking', 'Le...",0,"['One Survivor Remembers', 'Jim Dine: A Self-P...","['Braveheart', 'Batman Forever', 'A Little Pri...","['The Usual Suspects', 'Babe', 'Apollo 13', '1...","['Sense and Sensibility', 'Apollo 13', 'Babe',...","['The Postman (Il Postino)', 'Apollo 13', 'Bra...","['Dead Man Walking', 'Leaving Las Vegas', 'Cas...","['Apollo 13', 'Babe', 'Braveheart', 'Crimson T...","['Pocahontas', 'Dead Man Walking', 'Don Juan D...","['Braveheart', 'Batman Forever', 'Crimson Tide']","['Pocahontas', 'The American President', 'Sabr...",0,"['The Usual Suspects', 'Braveheart', 'Mighty A...","['Leaving Las Vegas', ""Mr. Holland's Opus"", 'N...","['Braveheart', 'My Family, Mi Familia', 'Roomm..."
7,"['When We Were Kings', 'The Line King: The Al ...","['The English Patient', 'The Birdcage', 'Evita...",0,"['Kolya', 'A Chef in Love', 'The Other Side of...",0,0,"['The English Patient', 'Evita', 'Independence...",0,0,"['Independence Day', 'Dragonheart', 'Twister']","['The English Patient', 'The Crucible', 'The M...",0,"['The English Patient', 'Fargo', 'Jerry Maguir...",0,0,"['Dear Diary', 'De Tripas, Corazon', 'Ernst & ...",0,"['The English Patient', 'Angels and Insects', ...","['Quest', 'Canhead', 'La Salla', ""Wat's Pig""]","['The English Patient', 'Fargo', 'The People v...",0,"[""Breathing Lessons: The Life and Work of Mark...","['T

In [10]:
genresNew = list(genresData.keys())
genresNew.append("Total")
oscarCats = pd.DataFrame(index=genresNew, columns=oscarNomsCleaned1.keys())
oscarCats = oscarCats.fillna(0.0)
oscarNomsCleaned1
for category in oscarNomsCleaned1:
    for year in range(5, len(oscarNomsCleaned1[category])+5):
        if oscarNomsCleaned1[category][year] == 0:
            continue
        for movie in ast.literal_eval(oscarNomsCleaned1[category][year]):
            print(movie)
            if len(movieData[movieData["original_title"] == movie]) < 1:
                continue
            genres = movieData["genre_names"][movieData[movieData["original_title"] == movie].index[0]]
            for genre in genres:
                oscarCats[category][genre] += 1
            oscarCats[category]["Total"] += 1
oscarCats


Maya Lin: A Strong Clear Vision
Complaints of a Dutiful Daughter
D-Day Remembered
Freedom on My Mind
A Great Day in Harlem
Anne Frank Remembered
The Battle over Citizen Kane
Fiddlefest--Roberta Tzavaras and Her East Harlem Violin Program
Hank Aaron: Chasing the Dream
Troublesome Creek: A Midwestern
When We Were Kings
The Line King: The Al Hirschfeld Story
Mandela
Suzanne Farrell: Elusive Muse
Tell the Truth and Run: George Seldes and the American Press
The Long Way Home
Ayn Rand: A Sense of Life
Colors Straight Up
4 Little Girls
Waco: The Rules of Engagement
The Last Days
Dancemaker
The Farm: Angola, U.S.A.
Lenny Bruce: Swear to Tell the Truth
Regret to Inform
One Day in September
Buena Vista Social Club
Genghis Blues
On the Ropes
Speaking in Strings
Into the Arms of Strangers: Stories of the Kindertransport
Legacy
Long Night's Journey into Day
Scottsboro: An American Tragedy
Sound and Fury
Murder on a Sunday Morning
Children Underground
LaLee's Kin: The Legacy of Cotton
Promises
War P

The Kite Runner
Michael Clayton
Ratatouille
3:10 to Yuma
Slumdog Millionaire
The Curious Case of Benjamin Button
Defiance
Milk
WALL-E
Up
Avatar
Fantastic Mr. Fox
The Hurt Locker
Sherlock Holmes
The Social Network
How to Train Your Dragon
Inception
The King's Speech
127 Hours
The Artist
The Adventures of Tintin
Hugo
Tinker Tailor Soldier Spy
War Horse
Life of Pi
Anna Karenina
Argo
Lincoln
Skyfall
Gravity
The Book Thief
Her
Philomena
Saving Mr. Banks
The Grand Budapest Hotel
The Imitation Game
Interstellar
Mr. Turner
The Theory of Everything
The Hateful Eight
Bridge of Spies
Carol
Sicario
Star Wars: The Force Awakens
La La Land
Jackie
Lion
Moonlight
Passengers
Speed
Clear and Present Danger
Forrest Gump
Legends of the Fall
The Shawshank Redemption
Apollo 13
Batman Forever
Braveheart
Crimson Tide
Waterworld
The English Patient
Evita
Independence Day
The Rock
Twister
Titanic
Air Force One
Con Air
Contact
L.A. Confidential
Saving Private Ryan
Armageddon
The Mask of Zorro
Shakespeare in Love

The Polar Express
Spider-Man 2
King Kong
Memoirs of a Geisha
War of the Worlds
Letters from Iwo Jima
Apocalypto
Blood Diamond
Flags of Our Fathers
Pirates of the Caribbean: Dead Man's Chest
The Bourne Ultimatum
No Country for Old Men
Ratatouille
There Will Be Blood
Transformers
The Dark Knight
Iron Man
Slumdog Millionaire
WALL-E
Wanted
The Hurt Locker
Avatar
Inglourious Basterds
Star Trek
Up
Inception
Toy Story 3
Tron: Legacy
True Grit
Unstoppable
Hugo
Drive
The Girl with the Dragon Tattoo
Transformers: Dark of the Moon
War Horse
Skyfall
Zero Dark Thirty
Argo
Django Unchained
Life of Pi
Gravity
All Is Lost
Captain Phillips
The Hobbit: The Desolation of Smaug
Lone Survivor
American Sniper
Birdman or (The Unexpected Virtue of Ignorance)
The Hobbit: The Battle of the Five Armies
Interstellar
Unbroken
Mad Max: Fury Road
The Martian
The Revenant
Sicario
Star Wars: The Force Awakens
Arrival
Deepwater Horizon
Hacksaw Ridge
La La Land
Sully
Talk to Her
Far from Heaven
Gangs of New York
My Big 

Lavatory - Lovestory
Oktapodi
Presto
This Way Up
Logorama
French Roast
Granny O'Grimm's Sleeping Beauty
The Lady and the Reaper (La Dama y la Muerte)
A Matter of Loaf and Death
The Lost Thing
Day & Night
The Gruffalo
Let's Pollute
Madagascar, carnet de voyage (Madagascar, a Journey Diary)
The Fantastic Flying Books of Mr. Morris Lessmore
Dimanche/Sunday
La Luna
A Morning Stroll
Wild Life
Paperman
Adam and Dog
Fresh Guacamole
Head over Heels
Maggie Simpson in "The Longest Daycare"
Mr. Hublot
Feral
Get a Horse!
Possessions
Room on the Broom
Feast
The Bigger Picture
The Dam Keeper
Me and My Moulton
A Single Life
Bear Story
Prologue
Sanjay’s Super Team
We Can’t Live without Cosmos
World of Tomorrow
Piper
Blind Vaysha
Borrowed Time
Pear Cider and Cigarettes
Pearl
Forrest Gump
Bullets over Broadway
Pulp Fiction
Quiz Show
Red
Braveheart
Babe
Dead Man Walking
Leaving Las Vegas
The Postman (Il Postino)
The English Patient
Fargo
The People vs. Larry Flynt
Secrets & Lies
Shine
Titanic
The Full Mo

The Last Station
The Lovely Bones
The Fighter
Winter's Bone
The Town
The Kids Are All Right
The King's Speech
Beginners
My Week with Marilyn
Moneyball
Warrior
Extremely Loud & Incredibly Close
Django Unchained
Argo
Silver Linings Playbook
The Master
Lincoln
Dallas Buyers Club
Captain Phillips
American Hustle
12 Years a Slave
The Wolf of Wall Street
Whiplash
The Judge
Boyhood
Birdman or (The Unexpected Virtue of Ignorance)
Foxcatcher
Bridge of Spies
The Big Short
The Revenant
Spotlight
Creed
Moonlight
Hell or High Water
Manchester by the Sea
Lion
Nocturnal Animals
Forrest Gump
The Madness of King George
Nobody's Fool
Quiz Show
The Shawshank Redemption
Sense and Sensibility
Apollo 13
Babe
Leaving Las Vegas
The Postman (Il Postino)
Sling Blade
The Crucible
The English Patient
Hamlet
Trainspotting
L.A. Confidential
Donnie Brasco
The Sweet Hereafter
Wag the Dog
The Wings of the Dove
Gods and Monsters
Out of Sight
Primary Colors
A Simple Plan
The Thin Red Line
The Cider House Rules
Election


Sweet and Lowdown
The Hurricane
Gladiator
Before Night Falls
Cast Away
Pollock
Quills
Training Day
A Beautiful Mind
I Am Sam
Ali
In the Bedroom
The Pianist
Adaptation
The Quiet American
Gangs of New York
About Schmidt
Mystic River
Pirates of the Caribbean: The Curse of the Black Pearl
House of Sand and Fog
Cold Mountain
Lost in Translation
Ray
Hotel Rwanda
Finding Neverland
The Aviator
Million Dollar Baby
Capote
Hustle & Flow
Brokeback Mountain
Walk the Line
Good Night, and Good Luck.
The Last King of Scotland
Blood Diamond
Half Nelson
Venus
The Pursuit of Happyness
There Will Be Blood
Michael Clayton
Sweeney Todd The Demon Barber of Fleet Street
In the Valley of Elah
Eastern Promises
Milk
The Visitor
Frost/Nixon
The Curious Case of Benjamin Button
The Wrestler
Crazy Heart
Up in the Air
A Single Man
Invictus
The Hurt Locker
The King's Speech
Biutiful
True Grit
The Social Network
127 Hours
The Artist
Demián Bichir
The Descendants
Tinker Tailor Soldier Spy
Moneyball
Lincoln
Silver Lining

,Documentary (Feature),Art Direction,Sound Mixing,Foreign Language Film,Makeup and Hairstyling,Music (Original Score),Sound,Special Achievement Award,Writing (Adapted Screenplay),Visual Effects,Actress in a Supporting Role,Writing (Screenplay Based on Material from Another Medium),Best Picture,Sound Editing,Writing (Original Screenplay),Short Film (Live Action),Animated Feature Film,Costume Design,Short Film (Animated),Directing,Production Design,Documentary (Short Subject),Cinematography,Actor in a Supporting Role,Writing (Screenplay Based on Material Previously Produced or Published),Music (Original Dramatic Score),Actress in a Leading Role,Film Editing,Music (Original Song),Sound Effects Editing,Music (Original Musical or Comedy Score),Special Achievement Award (Visual Effects),Writing (Screenplay Written Directly for the Screen),Actor in a Leading Role,Makeup
Foreign,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Music,6.0,5.0,4.0,0.0,0.0,2.0,3.0,0.0,1.0,0.0,9.0,0.0,4.0,1.0,2.0,2.0,2.0,10.0,1.0,4.0,2.0,0.0,5.0,1.0,0.0,0.0,5.0,7.0,14.0,0.0,1.0,0.0,3.0,6.0,2.0
Western,0.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,2.0,0.0,3.0,2.0,2.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Drama,11.0,59.0,41.0,27.0,4.0,72.0,26.0,0.0,63.0,26.0,103.0,0.0,123.0,37.0,61.0,14.0,4.0,85.0,9.0,94.0,20.0,2.0,90.0,103.0,33.0,18.0,105.0,90.0,52.0,8.0,8.0,0.0,30.0,101.0,26.0
Romance,0.0,23.0,6.0,7.0,0.0,22.0,6.0,0.0,18.0,6.0,32.0,0.0,28.0,4.0,17.0,2.0,4.0,39.0,1.0,20.0,5.0,2.0,23.0,19.0,8.0,3.0,45.0,17.0,31.0,2.0,7.0,0.0,8.0,22.0,10.0
Adventure,0.0,19.0,27.0,2.0,6.0,20.0,13.0,0.0,5.0,50.0,2.0,0.0,16.0,29.0,6.0,0.0,24.0,18.0,4.0,11.0,6.0,1.0,15.0,5.0,3.0,0.0,0.0,14.0,20.0,6.0,6.0,0.0,1.0,8.0,15.0
Documentary,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0
Horror,0.0,3.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0
Thriller,4.0,12.0,22.0,5.0,2.0,20.0,11.0,0.0,15.0,14.0,13.0,0.0,31.0,26.0,15.0,2.0,2.0,10.0,2.0,27.0,6.0,3.0,25.0,31.0,7.0,4.0,15.0,36.0,9.0,8.0,0.0,0.0,7.0,15.0,8.0
Animation,1.0,1.0,3.0,0.0,0.0,8.0,0.0,1.0,1.0,4.0,0.0,0.0,2.0,7.0,4.0,1.0,44.0,2.0,45.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,25.0,0.0,8.0,0.0,1.0,0.0,0.0


In [11]:
oscarCats1 = copy.deepcopy(oscarCats)
oscarCats1 = oscarCats1.drop(["Total"])
for column in oscarCats1:
    total = oscarCats[column]["Total"]
    for genre in genresData.keys():
        if total == 0:
            oscarCats[column][genre] = 0
        else:
            oscarCats1[column][genre] = float(oscarCats[column][genre])/float(total)
oscarCats1

,Documentary (Feature),Art Direction,Sound Mixing,Foreign Language Film,Makeup and Hairstyling,Music (Original Score),Sound,Special Achievement Award,Writing (Adapted Screenplay),Visual Effects,Actress in a Supporting Role,Writing (Screenplay Based on Material from Another Medium),Best Picture,Sound Editing,Writing (Original Screenplay),Short Film (Live Action),Animated Feature Film,Costume Design,Short Film (Animated),Directing,Production Design,Documentary (Short Subject),Cinematography,Actor in a Supporting Role,Writing (Screenplay Based on Material Previously Produced or Published),Music (Original Dramatic Score),Actress in a Leading Role,Film Editing,Music (Original Song),Sound Effects Editing,Music (Original Musical or Comedy Score),Special Achievement Award (Visual Effects),Writing (Screenplay Written Directly for the Screen),Actor in a Leading Role,Makeup
Foreign,0.048780,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000
Music,0.073171,0.063291,0.059701,0.000000,0.000000,0.022222,0.069767,0.0,0.014493,0.000000,0.082569,0.0,0.028369,0.014706,0.029412,0.086957,0.041667,0.099010,0.019231,0.037736,0.083333,0.000000,0.048544,0.009009,0.000000,0.000000,0.046729,0.063636,0.150538,0.000000,0.05,0.0,0.081081,0.055046,0.044444
Western,0.000000,0.012658,0.029851,0.000000,0.083333,0.022222,0.000000,0.0,0.014493,0.013158,0.018349,0.0,0.021277,0.029412,0.029412,0.000000,0.041667,0.019802,0.000000,0.009434,0.000000,0.000000,0.048544,0.027027,0.000000,0.000000,0.000000,0.009091,0.000000,0.000000,0.00,0.0,0.000000,0.009174,0.000000
Drama,0.134146,0.746835,0.611940,0.870968,0.333333,0.800000,0.604651,0.0,0.913043,0.342105,0.944954,0.0,0.872340,0.544118,0.897059,0.608696,0.083333,0.841584,0.173077,0.886792,0.833333,0.064516,0.873786,0.927928,0.868421,1.000000,0.981308,0.818182,0.559140,0.470588,0.40,0.0,0.810811,0.926606,0.577778
Romance,0.000000,0.291139,0.089552,0.225806,0.000000,0.244444,0.139535,0.0,0.260870,0.078947,0.293578,0.0,0.198582,0.058824,0.250000,0.086957,0.083333,0.386139,0.019231,0.188679,0.208333,0.064516,0.223301,0.171171,0.210526,0.166667,0.420561,0.154545,0.333333,0.117647,0.35,0.0,0.216216,0.201835,0.222222
Adventure,0.000000,0.240506,0.402985,0.064516,0.500000,0.222222,0.302326,0.0,0.072464,0.657895,0.018349,0.0,0.113475,0.426471,0.088235,0.000000,0.500000,0.178218,0.076923,0.103774,0.250000,0.032258,0.145631,0.045045,0.078947,0.000000,0.000000,0.127273,0.215054,0.352941,0.30,0.0,0.027027,0.073394,0.333333
Documentary,0.914634,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.019231,0.000000,0.000000,0.516129,0.000000,0.000000,0.000000,0.000000,0.000000,0.009091,0.043011,0.000000,0.00,0.0,0.000000,0.009174,0.000000
Horror,0.000000,0.037975,0.029851,0.032258,0.083333,0.011111,0.000000,0.0,0.000000,0.026316,0.000000,0.0,0.007092,0.029412,0.000000,0.000000,0.000000,0.009901,0.057692,0.009434,0.041667,0.032258,0.019417,0.018018,0.000000,0.000000,0.000000,0.009091,0.010753,0.000000,0.00,0.0,0.000000,0.009174,0.066667
Thriller,0.048780,0.151899,0.328358,0.161290,0.166667,0.222222,0.255814,0.0,0.217391,0.184211,0.119266,0.0,0.219858,0.382353,0.220588,0.086957,0.041667,0.099010,0.038462,0.254717,0.250000,0.096774,0.242718,0.279279,0.184211,0.222222,0.140187,0.327273,0.096774,0.470588,0.00,0.0,0.189189,0.137615,0.177778
Animation,0.012195,0.012658,0.044776,0.000000,0.000000,0.088889,0.000000,1.0,0.014493,0.052632,0.000000,0.0,0.014184,0.102941,0.058824,0.043478,0.916667,0.019802,0.865385,0.000000,0.000000,0.032258,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.268817,0.000000,0.40,0.0,0.027027,0.000000,0.000000


In [12]:
for category in oscarCats1:
    for genre in genresData.keys():
        oscarCats1[category][genre] -= totalRatios1[genre][0]
        
oscarCats1["Genre"] = oscarCats1.index

oscarCatsMelted = oscarCats1.melt(id_vars = ["Genre"])
oscarCatsMelted
#oscarCatsMelted = oscarCats.melt(id_vars = ["yearNames"])

,Genre,variable,value
0,Foreign,Documentary (Feature),0.048118
1,Music,Documentary (Feature),0.051366
2,Western,Documentary (Feature),-0.007304
3,Drama,Documentary (Feature),-0.172546
4,Romance,Documentary (Feature),-0.138067
...,...,...,...
695,Action,Makeup,-0.178252
696,Crime,Makeup,-0.051732
697,Family,Makeup,-0.173084
698,Science Fiction,Makeup,0.002250


In [13]:
altair.Chart(oscarCatsMelted).mark_rect().encode(
    x=altair.X('Genre:O', title="Genre"),
    y=altair.Y('variable:O', title="Category"),
    color=altair.Color('value:Q', scale=altair.Scale(scheme='redyellowgreen')),
    tooltip = [altair.Tooltip("value:Q", title="Difference")]
).interactive()

alt.Chart(...)

In [14]:

categories = list(oscarNomsCleaned)
#oscarNomsCleaned 
#titleGenres
genreFrame = pd.DataFrame(columns = genresData.keys())
oscarNomsCleaned.applymap(lambda x: str(x).strip("[]").split(","))
oscarNomsCleaned = pd.concat([oscarNomsCleaned, genreFrame.reindex(oscarNomsCleaned.index)], axis=1)
oscarNomsCleaned = oscarNomsCleaned.fillna(0.0)
totalSums = []
for i in range(5, 28):
    total = 0
    for category in categories:
        #print(category)
        if oscarNomsCleaned[category][i] == 0 or oscarNomsCleaned[category][i] == ['']:
            continue
        #print(oscarNomsCleaned[category][i])
        for nominee in ast.literal_eval(oscarNomsCleaned[category][i]):
            #print(nominee)
            if nominee not in titleGenres.keys():
                continue
            for genre in titleGenres[nominee]:
                #print(oscarNomsCleaned[genre["name"]])
                oscarNomsCleaned[genre["name"]][i] += 1
            total += 1
    totalSums.append(total)
for genre in genreFrame:
    for j in range(5, 28):
        oscarNomsCleaned[genre][j] = float((oscarNomsCleaned[genre][j])/totalSums[j-5])

oscarRatios = oscarNomsCleaned[genresData.keys()]
oscarRatios = oscarRatios.rename(lambda x: x+1989)
oscarRatios

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy
1994,0.000000,0.000000,0.024691,0.753086,0.345679,0.086420,0.037037,0.049383,0.197531,0.061728,0.123457,0.049383,0.012346,0.012346,0.000000,0.098765,0.222222,0.086420,0.000000,0.345679
1995,0.011628,0.011628,0.000000,0.720930,0.139535,0.046512,0.034884,0.023256,0.058140,0.093023,0.162791,0.116279,0.116279,0.000000,0.000000,0.209302,0.069767,0.209302,0.000000,0.232558
1996,0.000000,0.054348,0.000000,0.815217,0.304348,0.076087,0.032609,0.021739,0.173913,0.021739,0.108696,0.130435,0.010870,0.032609,0.000000,0.076087,0.108696,0.043478,0.054348,0.173913
1997,0.000000,0.000000,0.000000,0.642105,0.347368,0.084211,0.052632,0.000000,0.200000,0.010526,0.052632,0.010526,0.021053,0.157895,0.000000,0.273684,0.157895,0.010526,0.094737,0.242105
1998,0.011494,0.057471,0.000000,0.712644,0.241379,0.137931,0.034483,0.000000,0.080460,0.057471,0.448276,0.218391,0.091954,0.000000,0.000000,0.080460,0.057471,0.068966,0.057471,0.195402
1999,0.011364,0.090909,0.000000,0.761364,0.068182,0.045455,0.034091,0.011364,0.295455,0.045455,0.034091,0.000000,0.125000,0.125000,0.011364,0.102273,0.136364,0.034091,0.079545,0.181818
2000,0.000000,0.105263,0.000000,0.789474,0.092105,0.236842,0.026316,0.039474,0.210526,0.039474,0.013158,0.026316,0.013158,0.000000,0.000000,0.302632,0.092105,0.026316,0.039474,0.223684
2001,0.011628,0.093023,0.000000,0.523256,0.197674,0.209302,0.034884,0.000000,0.255814,0.104651,0.104651,0.093023,0.220930,0.127907,0.000000,0.255814,0.034884,0.093023,0.081395,0.162791
2002,0.000000,0.000000,0.011494,0.747126,0.172414,0.126437,0.034483,0.000000,0.103448,0.091954,0.114943,0.091954,0.114943,0.011494,0.000000,0.137931,0.333333,0.068966,0.045977,0.264368
2003,0.000000,0.011236,0.000000,0.539326,0.089888,0.314607,0.044944,0.022472,0.101124,0.101124,0.123596,0.044944,0.191011,0.067416,0.000000,0.235955,0.112360,0.078652,0.011236,0.067416


In [15]:
oscarDifferences = oscarRatios.subtract(yearsRatios1)
oscarDifferences["yearNames"] = oscarDifferences.index
oscarDifferencesMelted = oscarDifferences.melt(id_vars = ["yearNames"])
oscarDifferencesOver = oscarDifferencesMelted
altair.Chart(oscarDifferencesMelted).mark_rect().encode(
    x=altair.X('yearNames:O', title="Year"),
    y=altair.Y('variable:O', title="Genre"),
    color=altair.Color('value:Q', scale=altair.Scale(scheme='redyellowgreen')),
    tooltip = [altair.Tooltip("value:Q", title="Difference")]
).interactive()

alt.Chart(...)

In [16]:
oscarDifferenceMeans = pd.DataFrame(oscarDifferences.mean().to_dict(),index=[1])
overRep = []
middle = []
underRep = []
for col in oscarDifferenceMeans:
    if oscarDifferenceMeans[col][1] > 5:
        overRep.append(col)
    elif oscarDifferenceMeans[col][1] < -5:
        underRep.append(col)
    else:
        middle.append(col)

print(overRep)
middle.append("yearNames")
underRep.append("yearNames")
oscarDifferenceMeans
oscarsOver = oscarDifferences[overRep]
oscarsUnder = oscarDifferences[underRep]
oscarsMiddle = oscarDifferences[middle]
oscarsOverMelted = oscarsOver.melt(id_vars = ["yearNames"])
oscarsUnderMelted = oscarsUnder.melt(id_vars = ["yearNames"])
oscarsMiddleMelted = oscarsMiddle.melt(id_vars = ["yearNames"])
altair.Chart(oscarsMiddleMelted).mark_line().encode(
  x='yearNames:O',
  y='value:Q',
  color='variable'
)
oscarRatios["yearNames"] = oscarRatios.index
yearsRatios1["yearNames"] = yearsRatios1.index
oscarRatiosMelted = oscarRatios.melt(id_vars = ["yearNames"])
genreRatiosMelted = yearsRatios1.melt(id_vars = ["yearNames"])
altair.Chart(oscarDifferencesMelted[oscarDifferencesMelted["variable"] == "Fantasy"]).mark_line().encode(
   x ='yearNames:O',
   y='value:Q',
   color="variable"
)
altair.Chart(oscarRatiosMelted[oscarRatiosMelted["variable"] == "Fantasy"]).mark_line().encode(
    x='yearNames:O',
    y='value:Q',
    color="variable"
)
altair.Chart(genreRatiosMelted[genreRatiosMelted["variable"] == "Fantasy"]).mark_line().encode(
     x='yearNames:O',
     y='value:Q',
     color="variable"
 )

['yearNames']


alt.Chart(...)

In [34]:
movieData["release_month"] = movieData["release_date"].apply(lambda x: str(x)[5:7])
monthRevenues = movieData.groupby("release_month")["revenue"].sum()
monthRevenues
monthMovies = movieData.groupby("release_month")
monthMovies.describe()
movieData = movieData[movieData.release_year.map(int) > 1990]
movieData = movieData.fillna(0.0)
movieData

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,genre_names,release_month
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,Toy Story,False,7.7,5415.0,1995,"[Animation, Comedy, Family]",10
1,False,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",0,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[Adventure, Fantasy, Family]",12
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[Romance, Comedy]",12
3,False,0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[Comedy, Drama, Romance]",12
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",0,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,[Comedy],02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45459,False,0,0,"[{'id': 878, 'name': 'Science Fiction'}]",0,222848,tt0112613,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,0.661558,/4lF9LH0b0Z1X94xGK9IOzqEW6k1.jpg,"[{'name': 'Concorde-New Horizons', 'id': 4688}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,Caged Heat 3000,False,3.5,1.0,1995,[Science Fiction],01
45460,False,0,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",0,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,"[{'name': 'Westdeutscher Rundfunk (WDR)', 'id'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,Robin Hood,False,5.7,26.0,1991,"[Drama, Action, Romance]",05
45462,False,0,0,"[{'id': 18, 'name': 'Drama'}]",0,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241

In [26]:
monthMovies

In [39]:
movieData["genre_names"] = movieData["genre_names"].apply(lambda x: ast.literal_eval(str(x)))
movieData["release_year"] = movieData["release_year"].apply(lambda x: int(x))
movieData["release_month"] = movieData["release_month"].apply(lambda x: int(x))
newGenres = list(genresData.keys())
newGenres.append("month")
newGenres.append("Total")
months = []
for i in range(1, 13):
    months.append(i)
seasonality = pd.DataFrame(columns=newGenres, index = months)
for genre in newGenres:
    for i in range(1, 13):
        seasonality[genre] = 0.0
    seasonality["Total"] = 0.0
seasonality["month"] = seasonality.index
countsSeasonality = copy.deepcopy(seasonality)
seasonality
for i in range(0, len(movieData["original_title"])):
    if i not in movieData.index:
        continue
    if movieData["release_year"][i] not in range(1994, 2015):
        continue
    if movieData["revenue"][i] <= 0:
        continue
    if movieData["release_month"][i] <= 0:
        continue
    for genre in movieData["genre_names"][i]:
        seasonality[genre][movieData["release_month"][i]] += movieData["revenue"][i]
    seasonality["Total"][movieData["release_month"][i]] += movieData["revenue"][i]
newSeasonality = copy.deepcopy(seasonality)
newSeasonality
    
        
    

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,month,Total
1,1662895.0,1.423445e+08,10502464.0,3.057163e+09,1.409985e+09,1.681456e+09,220756696.0,2.313958e+09,3.280364e+09,2.431789e+08,4.620115e+08,3.862141e+08,1.427950e+09,1.150351e+09,0.0,4.604405e+09,1.556104e+09,1.478463e+09,1.567952e+09,4.135806e+09,1,1.116512e+10
2,13150076.0,5.121749e+08,107082675.0,7.855492e+09,4.340890e+09,3.667413e+09,201790793.0,1.802445e+09,5.437608e+09,1.368675e+09,2.552166e+08,2.487429e+08,2.682339e+09,1.594952e+09,0.0,4.371179e+09,3.073998e+09,2.554329e+09,1.106409e+09,6.206168e+09,2,1.725639e+10
3,1275162.0,2.891333e+08,353828053.0,6.675022e+09,3.680171e+09,1.196501e+10,85707138.0,1.204595e+09,5.475411e+09,4.999972e+09,1.292238e+08,7.619814e+08,6.243075e+09,1.534526e+09,42000000.0,7.339441e+09,3.433826e+09,7.948819e+09,5.866786e+09,9.120326e+09,3,2.593255e+10
4,0.0,1.856365e+08,41060396.0,5.416632e+09,3.652642e+09,1.005991e+10,168596504.0,1.161116e+09,6.063004e+09,1.036431e+09,2.197586e+08,2.604594e+08,3.228124e+09,1.905757e+09,0.0,1.193857e+10,3.461251e+09,1.747164e+09,6.226354e+09,6.720920e+09,4,2.218151e+10
5,1477022.0,2.721238e+08,225008697.0,8.347052e+09,6.043096e+09,2.252650e+10,73672179.0,7.085800e+08,8.908913e+09,6.723502e+09,1.068004e+09,1.516649e+09,1.326175e+10,1.966328e+09,0.0,2.147248e+10,1.831938e+09,1.001738e+10,1.009947e+10,1.150935e+10,5,4.243455e+10
6,302001.0,1.864758e+08,259400711.0,1.121658e+10,4.538706e+09,2.199736e+10,40588399.0,8.755662e+08,9.513245e+09,1.267096e+10,4.721103e+08,4.125601e+08,1.119187e+10,3.000365e+09,0.0,2.199078e+10,5.055350e+09,1.553619e+10,1.172389e+10,1.890478e+10,6,4.689352e+10
7,20460000.0,4.379165e+08,265143478.0,8.330652e+09,5.119615e+09,1.559729e+10,78331350.0,1.808957e+09,1.087550e+10,4.565841e+09,1.112451e+09,7.938445e+08,9.648448e+09,2.994385e+09,0.0,1.684784e+10,5.124392e+09,8.448259e+09,9.363268e+09,1.385885e+10,7,3.744933e+10
8,28426856.0,2.481192e+08,82598734.0,8.400356e+09,3.124269e+09,4.777560e+09,23714783.0,2.109572e+09,9.861081e+09,6.930578e+08,4.521118e+08,3.628774e+08,2.244032e+09,2.735562e+09,0.0,9.492009e+09,3.421234e+09,2.171317e+09,3.916817e+09,7.147794e+09,8,2.196307e+10
9,24812366.0,6.515731e+08,109737274.0,1.149101e+10,3.445979e+09,2.207022e+09,150180276.0,2.152193e+09,8.097220e+09,1.980569e+09,1.103778e+09,4.280594e+08,1.264626e+09,2.694758e+09,0.0,5.739125e+09,5.234521e+09,2.231794e+09,3.269574e+09,6.221866e+09,9,2.177006e+10
10,3875540.0,4.416318e+08,5048693.0,7.745881e+09,2.861853e+09,7.848470e+09,268514721.0,3.789280e+09,7.294304e+09,3.759176e+09,4.959670e+08,7.710469e+08,3.617879e+09,2.732839e+09,0.0,8.341359e+09,3.438135e+09,6.027599e+09,2.129032e+09,8.433125e+09,10,2.501150e+10


In [40]:
for genre in genresData.keys():
    for i in range(1, 13):
        seasonality[genre][i] /= seasonality["Total"][i]
seasonality

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,month,Total
1,0.000149,0.012749,0.000941,0.273814,0.126285,0.150599,0.019772,0.207249,0.293805,0.021780,0.041380,0.034591,0.127894,0.103031,0.00000,0.412392,0.139372,0.132418,0.140433,0.370422,1,1.116512e+10
2,0.000762,0.029680,0.006205,0.455222,0.251553,0.212525,0.011694,0.104451,0.315107,0.079314,0.014790,0.014415,0.155440,0.092427,0.00000,0.253308,0.178137,0.148022,0.064116,0.359645,2,1.725639e+10
3,0.000049,0.011149,0.013644,0.257399,0.141913,0.461390,0.003305,0.046451,0.211140,0.192807,0.004983,0.029383,0.240743,0.059174,0.00162,0.283020,0.132414,0.306519,0.226233,0.351694,3,2.593255e+10
4,0.000000,0.008369,0.001851,0.244196,0.164671,0.453527,0.007601,0.052346,0.273336,0.046725,0.009907,0.011742,0.145532,0.085916,0.00000,0.538222,0.156042,0.078767,0.280700,0.302997,4,2.218151e+10
5,0.000035,0.006413,0.005302,0.196704,0.142410,0.530853,0.001736,0.016698,0.209945,0.158444,0.025168,0.035741,0.312523,0.046338,0.00000,0.506014,0.043171,0.236067,0.238001,0.271226,5,4.243455e+10
6,0.000006,0.003977,0.005532,0.239192,0.096787,0.469092,0.000866,0.018671,0.202869,0.270207,0.010068,0.008798,0.238665,0.063982,0.00000,0.468951,0.107805,0.331308,0.250011,0.403143,6,4.689352e+10
7,0.000546,0.011694,0.007080,0.222451,0.136708,0.416490,0.002092,0.048304,0.290406,0.121920,0.029705,0.021198,0.257640,0.079958,0.00000,0.449884,0.136835,0.225592,0.250025,0.370069,7,3.744933e+10
8,0.001294,0.011297,0.003761,0.382476,0.142251,0.217527,0.001080,0.096051,0.448985,0.031556,0.020585,0.016522,0.102173,0.124553,0.00000,0.432181,0.155772,0.098862,0.178337,0.325446,8,2.196307e+10
9,0.001140,0.029930,0.005041,0.527836,0.158290,0.101379,0.006898,0.098860,0.371943,0.090977,0.050702,0.019663,0.058090,0.123783,0.00000,0.263625,0.240446,0.102517,0.150187,0.285799,9,2.177006e+10
10,0.000155,0.017657,0.000202,0.309693,0.114421,0.313794,0.010736,0.151502,0.291638,0.150298,0.019830,0.030828,0.144649,0.109263,0.00000,0.333501,0.137462,0.240993,0.085122,0.337170,10,2.501150e+10


In [46]:
seasonalityMelted = seasonality.melt(id_vars = ["month", "Total"])
seasonalityMelted1 = seasonalityMelted[0:4999]
altair.Chart(seasonalityMelted1).mark_rect().encode(
    x=altair.X('month:N', title="Month"),
    y=altair.Y('variable:O', title="Genre"),
    color=altair.Color('value:Q', title="Intensity"),
    tooltip = [altair.Tooltip("value:Q", title="Dominance")]
)

alt.Chart(...)

In [47]:
countsSeasonality

,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,month,Total
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0


In [49]:
for j in range(0, len(movieData["revenue"])):
    if j not in movieData.index.values:
        continue
    if movieData["release_year"][j] in range(1994, 2015):
        countsSeasonality["Total"][movieData["release_month"][j]] += 1
        for genre in movieData["genre_names"][j]:
            countsSeasonality[genre][movieData["release_month"][j]] += 1
for i in range(1, 13):
    for genre in genresData.keys():
        countsSeasonality[genre][i] /= countsSeasonality["Total"][i]
countsSeasonality

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,month,Total
1,0.048479,0.024715,0.008555,0.432510,0.145437,0.055133,0.161122,0.103612,0.194867,0.031844,0.019011,0.018536,0.041350,0.051331,0.015209,0.153992,0.085076,0.047529,0.066540,0.285646,1,4208.0
2,0.050704,0.027230,0.008451,0.506103,0.177465,0.077934,0.085446,0.092958,0.209390,0.049765,0.033803,0.021596,0.052582,0.061972,0.015023,0.162441,0.110798,0.071362,0.056338,0.301408,2,2130.0
3,0.034063,0.026764,0.011354,0.433090,0.159773,0.079481,0.125710,0.089213,0.184104,0.047851,0.024331,0.017843,0.042174,0.055150,0.012976,0.153285,0.111922,0.072182,0.073804,0.325223,3,2466.0
4,0.041459,0.021559,0.006633,0.446103,0.150083,0.092869,0.126036,0.098673,0.212272,0.025705,0.025705,0.015755,0.045605,0.059701,0.013267,0.178275,0.114428,0.061360,0.077114,0.337479,4,2412.0
5,0.053989,0.024980,0.011281,0.568090,0.181305,0.078163,0.085415,0.079774,0.207091,0.033038,0.040290,0.028203,0.051571,0.057212,0.014504,0.156326,0.087027,0.056406,0.075745,0.265915,5,2482.0
6,0.050864,0.033589,0.011516,0.436660,0.173704,0.086372,0.136276,0.076775,0.184261,0.053743,0.019194,0.017274,0.049904,0.048944,0.007678,0.162188,0.093090,0.074856,0.074856,0.343570,6,2084.0
7,0.033259,0.028825,0.005543,0.369180,0.139690,0.135255,0.123060,0.110865,0.222838,0.080931,0.026608,0.015521,0.089800,0.047672,0.015521,0.198448,0.100887,0.105322,0.100887,0.337029,7,1804.0
8,0.045059,0.029249,0.007905,0.470356,0.157312,0.081423,0.086166,0.127273,0.244269,0.046640,0.022925,0.012648,0.041897,0.060079,0.010277,0.179447,0.113834,0.075099,0.058498,0.342292,8,2530.0
9,0.053571,0.034821,0.004911,0.572321,0.164286,0.051786,0.104464,0.082143,0.205357,0.029911,0.040179,0.027232,0.039732,0.055357,0.005357,0.125446,0.104464,0.031696,0.052232,0.281250,9,4480.0
10,0.037747,0.035424,0.003484,0.463415,0.136469,0.062137,0.123693,0.161440,0.224158,0.040070,0.023810,0.013937,0.050523,0.070848,0.016841,0.122532,0.088269,0.064460,0.062137,0.284553,10,3444.0


In [54]:
countsSeasonalityMelted = countsSeasonality.melt(id_vars = ["month", "Total"])
countsSeasonalityMelted1 = countsSeasonalityMelted[0:5000]
altair.Chart(countsSeasonalityMelted1).mark_rect().encode(
    x=altair.X('month:N', title="Month"),
    y=altair.Y('variable:O', title="Genre"),
    color=altair.Color('value:Q', title="Intensity"),
    tooltip = [altair.Tooltip("value:Q", title="Release Count Dominance")]
)

alt.Chart(...)

In [55]:
newSeasonality

        

,Foreign,Music,Western,Drama,Romance,Adventure,Documentary,Horror,Thriller,Animation,History,War,Fantasy,Mystery,TV Movie,Action,Crime,Family,Science Fiction,Comedy,month,Total
1,1662895.0,1.423445e+08,10502464.0,3.057163e+09,1.409985e+09,1.681456e+09,220756696.0,2.313958e+09,3.280364e+09,2.431789e+08,4.620115e+08,3.862141e+08,1.427950e+09,1.150351e+09,0.0,4.604405e+09,1.556104e+09,1.478463e+09,1.567952e+09,4.135806e+09,1,1.116512e+10
2,13150076.0,5.121749e+08,107082675.0,7.855492e+09,4.340890e+09,3.667413e+09,201790793.0,1.802445e+09,5.437608e+09,1.368675e+09,2.552166e+08,2.487429e+08,2.682339e+09,1.594952e+09,0.0,4.371179e+09,3.073998e+09,2.554329e+09,1.106409e+09,6.206168e+09,2,1.725639e+10
3,1275162.0,2.891333e+08,353828053.0,6.675022e+09,3.680171e+09,1.196501e+10,85707138.0,1.204595e+09,5.475411e+09,4.999972e+09,1.292238e+08,7.619814e+08,6.243075e+09,1.534526e+09,42000000.0,7.339441e+09,3.433826e+09,7.948819e+09,5.866786e+09,9.120326e+09,3,2.593255e+10
4,0.0,1.856365e+08,41060396.0,5.416632e+09,3.652642e+09,1.005991e+10,168596504.0,1.161116e+09,6.063004e+09,1.036431e+09,2.197586e+08,2.604594e+08,3.228124e+09,1.905757e+09,0.0,1.193857e+10,3.461251e+09,1.747164e+09,6.226354e+09,6.720920e+09,4,2.218151e+10
5,1477022.0,2.721238e+08,225008697.0,8.347052e+09,6.043096e+09,2.252650e+10,73672179.0,7.085800e+08,8.908913e+09,6.723502e+09,1.068004e+09,1.516649e+09,1.326175e+10,1.966328e+09,0.0,2.147248e+10,1.831938e+09,1.001738e+10,1.009947e+10,1.150935e+10,5,4.243455e+10
6,302001.0,1.864758e+08,259400711.0,1.121658e+10,4.538706e+09,2.199736e+10,40588399.0,8.755662e+08,9.513245e+09,1.267096e+10,4.721103e+08,4.125601e+08,1.119187e+10,3.000365e+09,0.0,2.199078e+10,5.055350e+09,1.553619e+10,1.172389e+10,1.890478e+10,6,4.689352e+10
7,20460000.0,4.379165e+08,265143478.0,8.330652e+09,5.119615e+09,1.559729e+10,78331350.0,1.808957e+09,1.087550e+10,4.565841e+09,1.112451e+09,7.938445e+08,9.648448e+09,2.994385e+09,0.0,1.684784e+10,5.124392e+09,8.448259e+09,9.363268e+09,1.385885e+10,7,3.744933e+10
8,28426856.0,2.481192e+08,82598734.0,8.400356e+09,3.124269e+09,4.777560e+09,23714783.0,2.109572e+09,9.861081e+09,6.930578e+08,4.521118e+08,3.628774e+08,2.244032e+09,2.735562e+09,0.0,9.492009e+09,3.421234e+09,2.171317e+09,3.916817e+09,7.147794e+09,8,2.196307e+10
9,24812366.0,6.515731e+08,109737274.0,1.149101e+10,3.445979e+09,2.207022e+09,150180276.0,2.152193e+09,8.097220e+09,1.980569e+09,1.103778e+09,4.280594e+08,1.264626e+09,2.694758e+09,0.0,5.739125e+09,5.234521e+09,2.231794e+09,3.269574e+09,6.221866e+09,9,2.177006e+10
10,3875540.0,4.416318e+08,5048693.0,7.745881e+09,2.861853e+09,7.848470e+09,268514721.0,3.789280e+09,7.294304e+09,3.759176e+09,4.959670e+08,7.710469e+08,3.617879e+09,2.732839e+09,0.0,8.341359e+09,3.438135e+09,6.027599e+09,2.129032e+09,8.433125e+09,10,2.501150e+10


In [65]:
ratios = copy.deepcopy(newSeasonality)
ratios1 = copy.deepcopy(newSeasonality)
for genre in genresData.keys():
    for i in range(1, 13):
        ratios1[genre][i] /= ratios[genre].sum()
for i in range(1, 13):
    ratios1["Total"][i] /= ratios["Total"].sum()
ratios1
ratios1 = ratios1.drop(columns = ["TV Movie", "Music"])

/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/shivasaravanan/.local/share/virtualenvs/shivasaravanan-YA336bZY/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [68]:
ratiosMelted = ratios1.melt(id_vars = ["month"])
altair.Chart(ratiosMelted).mark_rect().encode(
    x=altair.X('month:N', title="Month"),
    y=altair.Y('variable:O', title="Genre"),
    color=altair.Color('value:Q', title="Intensity"),
    tooltip = [altair.Tooltip("value:Q", title="Reliance")]
).interactive()

alt.Chart(...)

In [25]:
ratios1["Western"]

0                0
1                0
2                0
3      1.52404e+07
4                0
          ...     
247              0
248              0
249              0
250              0
251              0
Name: Western, Length: 252, dtype: object